<a href="https://colab.research.google.com/github/Dntfreitas/inteligencia-artificial/blob/main/Pr%C3%A1tica/chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas necessárias

In [ ]:
# Instalação das bibliotecas necessárias
!pip install openai==0.28
!pip install python-dotenv
!pip install gradio

In [ ]:
import os
import pprint

import gradio as gr
import openai
from IPython.display import display, Markdown
from dotenv import load_dotenv, find_dotenv

# Configuração da API por variáveis de ambiente

A chave da API pode ser obtida em [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys).

A chave deverá depois ser guardada num ficheiro chamado, por exemplo, `.env` com o seguinte conteúdo:

```
OPENAI_API_KEY=chave-da-api
```

In [ ]:
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

# Funções auxiliares

O modelo a ser utilizado é o `gpt-3.5-turbo`. Contudo, podem ser utilizados outros modelos. A lista de modelos disponíveis pode ser obtida através do comando `openai.Engine.list()`.

A temperatura é um parâmetro que controla a aleatoriedade das respostas. Quanto maior for a temperatura, mais aleatórias serão as respostas. O valor por omissão é 0, pelo que as respostas serão sempre as mesmas.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", history=None, temperature=0):
    if history is None or len(history) == 0:
        history = [
            {
                "role": "system",
                "content": "You are an assitant..."  # persona que o bot assume
            }
        ]
    history.append({
        "role": "user",
        "content": prompt
    })
    response = openai.ChatCompletion.create(
        model=model,
        messages=history,
        temperature=temperature,

    )
    completion = response.choices[0].message["content"]
    history.append({
        "role": "system",
        "content": completion
    })
    return completion, history


In [ ]:
def print_as_md(completion):
    display(Markdown(completion))


# Exemplo de utilização

In [ ]:
completion, _ = get_completion("What is a neural network?")
print_as_md(completion)

**Tente, agora, alterar o prompt para obter respostas diferentes, alterar a temperatura para perceber como esta afeta as respostas e alterar o modelo para perceber como este afeta as respostas.**

## Criação de um chatbot

Para a criação de um chabot, precisamos de guardar o contexto, isto é, o histórico de mensagens entre o utilizador e o chatbot. Desta forma, o chatbot pode dar respostas mais adequadas ao contexto da conversa.

In [ ]:
history = None

In [ ]:
completion, history = get_completion("What is a neural network?", history=history)
print_as_md(completion)

In [ ]:
completion, history = get_completion("Hum... explain me as if I was a 5 year old.", history=history)
print_as_md(completion)

In [ ]:
pprint.pprint(history)

### Vamos, agora, criar uma app para interagir com o chatbot

Para tal, vamos usar a biblioteca [Gradio](https://www.gradio.app/), que permite criar interfaces para modelos de Machine Learning, neste caso, para o nosso chatbot.

In [ ]:
def chat_wrapper(prompt, history):
    history_arg = [
        {
            "role": "system",
            "content": "You are an assitant..."  # persona que o bot assume
        }
    ]

    # Since the history is a list of lists, such as [[user_message, bot_message], ...], we need to convert it to a list of dictionaries, such as [{"role": "user", "content": user_message}, {"role": "system", "content": bot_message}, ...]
    for i in range(len(history)):
        user_message = history[i][0]
        bot_message = history[i][1]

        history_arg.append({
            "role": "user",
            "content": user_message
        })

        history_arg.append({
            "role": "system",
            "content": bot_message
        })

    history_arg.append({
        "role": "user",
        "content": prompt
    })

    completion, _ = get_completion(prompt, history=history_arg)

    return completion


In [ ]:
chat = gr.ChatInterface(
    chat_wrapper
)

In [ ]:
chat.launch(share=True)